# PowerStatistics


### Coisa muitoo bacana
Coeficiente de uma equipa num ficheiro da Liga = Coeficiente de uma equipa num ficheiro com tds as ligas

In [1]:
import pandas as pd 
import numpy as np
import datetime as dt
import xlwings as xw
import os
import openpyxl as op

In [2]:
prem = pd.read_excel('PLTeste.xlsx',0)
lp=pd.read_excel('LPTeste.xlsx',0)

In [3]:
#damos qnts semanas queremos de jogos e a função devolve uma DataFrame com essas x últimas jornadas/semanas
def last_games(week,path,sheet=0):
    file=pd.read_excel(path+'.xlsx',sheet)
    g=len(file.index)
    d=file.iloc[g-1]['Date']
    weekz=date_week(d.year,d)
    found=False
    i=1
    while i<g and not(found):
        day_=file.iloc[g-1-i]['Date']
        if weekz-(date_week(day_.year,day_))>=week: #se estamos na semana 10, e queremos as últimas 5 jornadas/semanas, queremos 6-10
            found=True
        else:
            i+=1
    return file[(g-i):(g)]

#não rende muito
#Há equipas q perdem 1 jogo e são mais prejudicadas do que equipas que empatem 3 jogos

In [4]:
def fix_week(x):
    return (((x.isoweekday()+2)%7)+1)

def date_week(year,date):
    import datetime as dt
    if isinstance(date,str):
        day_=dt.datetime.strptime(date,"%d/%m/%Y")
    elif isinstance(date,dt.datetime):
        day_=date
    else:
        print("Erro! Not Datetime nor string.")
    first_day=dt.datetime(year,1,((8-fix_week(dt.datetime.strptime("01/01/"+str(year),"%d/%m/%Y")))%7)+1) #primeiro dia do ano das apostas
    take=first_day.day #dias a tirar 
    act_day=(day_.timetuple().tm_yday)-take #dia do ano das apostas do primeiro dia da folha de excel
    final=(act_day//7)+1
    if final>0:
        return final
    else:
        return 52


In [10]:
#dá uma lista com tds os clubes de uma época de uma liga
#teams é o número de equipas q tem a liga
def findall_clubs(path,teams):
    import pandas as pd
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    counter=0
    i=0
    done=False
    while i<len(file.index) and not(done):
        if i==teams+50:
            print('Está a ver demasiadas linhas, tem erro')
        if file.iloc[i]['HomeTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['HomeTeam']]
            counter+=1
        if file.iloc[i]['AwayTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['AwayTeam']]
            counter+=1
        if counter==teams:
            done=True
        i+=1
    return lista_c 

In [6]:
#theta vetor coluna
#X, matriz vetores linhas
def pred2(theta,X):
    return np.dot(X,theta)

def h(theta,X):
    e=np.e
    return 1/(1+(e**(-pred2(theta,X))))

#y é vetor linha
def LogR_cost(X,y,theta):
    m=len(y)
    return (-1/m)*(np.dot(y,np.log(h(theta,X)))+np.dot((1-y),(np.log(1-h(theta,X)))))

def LogR_grad(X,y,theta):
    m=len(y)
    return (1/m)*(np.dot(X.T,(h(theta,X)-(y.T))))

def gradient_descent_logit(panda_df,learning_rate=0.01,iterations=6000):
    import numpy as np
    X=adapt_X(panda_df,logit=True)[0]
    y=adapt_X(panda_df,logit=True)[1]
    m=len(y)
    dim=len(X[0])
    #print('Dim',dim)
    theta=np.random.randn(dim,1) #assim não temos de definir theta
    for it in range(iterations):
        cost=LogR_cost(X,y,theta)
        #print(X,y,theta)
        #print('________')
        #print(cost)
        theta=theta-LogR_grad(X,y,theta)*learning_rate
    return theta, cost
#Nota, parece q o cost às vezes dá erro, pq o valor da probabilidade chega a ser 1, dps faz-se Log(1-(1))=Log(0)=-inf
#Mas parece q a função não falha


#esta função ainda é beta-version, mas diz-nos a prob através do gráfico q aprendeu
#input_ pode ser array de linhas, cada linha, uma variável indep. 
def LogReg(input_,panda_df,learning_rate=0.01,iterations=6000):
    e=np.e
    G=gradient_descent_logit(panda_df,learning_rate=0.01,iterations=6000)
    t=G[0][0]+(np.dot(G[0][1:],input_))
    return 1/(1+(e**(-t)))

# | The Magic |
- X, matriz que já sabemos q tem de ser a transposta
- theta, matriz linha
- y, matriz linha

###### Atenção
Vamos ter de fazer uma transposta no cálculo

In [7]:
#devolve uma lista de -1,0,1 se o clube joga fora,não joga, joga em casa
#panda já é o ficheiro, NÃO é o path
def PDF_aux1(panda,club,H_coef=1):
    q=[]
    for i in range(len(panda.index)):
        if panda.iloc[i]['HomeTeam']==club:
            q+=[H_coef]
        elif panda.iloc[i]['AwayTeam']==club:
            q+=[-1]
        else:
            q+=[0]
    return q

#cria a dataframe do Power Stats para usar na LogitReg
#c_list é a lista de clubes
def PowerDF(panda_df,c_list,H_coef=1):
    pdf=pd.DataFrame()
    for club in c_list:
        pdf[club]=PDF_aux1(panda_df,club,H_coef=H_coef)
    new_df=panda_df['Goal_Diff'].reset_index()
    pdf['target']=new_df['Goal_Diff'] #assim já não dá erro qnd o índice é diferente
    return pdf

#H_coef existe pq os clubes que jogam em casa costumam ter a 'Home advantage'
#Podemos dizer H_coef=0.8, para tentar equilibrar, exemplo

In [8]:
def pred3(X,theta):
    return np.dot(theta,X) #para ter uma melhor noção, ver o apontamento do caderno

#Cost_function do Power Stats
def PS_cost(X,y,theta):
    return sum(np.square(y-pred3(X,theta))[0])

#Gradiente do Power Stats
#ver apontamentos para ver os cálculos 
def PS_grad(X,y,theta):
    return (np.dot((2*X),((y-pred3(X,theta)).T))).T

def PS_Gdescent(panda_df,learning_rate=0.01,iterations=10000):
    import numpy as np
    preX=np.array(panda_df.drop('target',axis=1))
    y=np.array([panda_df['target']])
    X=preX.T
    #X=adapt_X(panda_df,logit=True)[0]
    #y=adapt_X(panda_df,logit=True)[1]
    dim=len(X)
    #print('Dim',dim)
    #theta=np.random.randn(1,dim) #assim não temos de definir theta
    theta=np.ones((1,dim),dtype=int)
    for it in range(iterations):
        cost=PS_cost(X,y,theta)
        #print(X,y,theta)
        #print('________')
        #print(cost)
        theta=theta-(learning_rate*(-(PS_grad(X,y,theta))))
    return np.exp(theta), cost

In [249]:
#nicez=PS_Gdescent(qq,iterations=10000)
#Lpool=nicez[0][0][0]
#Norwich=nicez[0][0][1]
#Mcity=nicez[0][0][3]
#print(Lpool,Mcity)
#print(Lpool/Mcity)
#print(Lpool/Norwich)


_____________

In [9]:
overall=pd.read_excel('19_20.xlsx',0)

In [13]:
ccall=findall_clubs('19_20',134)
#ccall

In [14]:
allPS=PowerDF(overall,ccall)

In [15]:
allPS2=PS_Gdescent(allPS,iterations=10000)

In [16]:
df_all=pd.DataFrame(allPS2[0],columns=ccall)

In [18]:
pd.set_option('display.max_columns',150)

In [254]:
#bund5=last_games(5,'BundTeste')
#bund5['Goal_Diff']

In [271]:
cc3=findall_clubs('BundTeste',18)
bundPS=PowerDF(bund,cc3)
bundPS2=PS_Gdescent(bundPS,iterations=10000)

In [272]:
df_Bund=pd.DataFrame(bundPS2[0],columns=cc3)

In [20]:
df_all

,Zwolle,Willem II,FC Emmen,Groningen,Vitesse,Ajax,Twente,PSV Eindhoven,VVV Venlo,Waalwijk,Heracles,Heerenveen,Feyenoord,Sparta Rotterdam,Den Haag,Utrecht,AZ Alkmaar,For Sittard,Monaco,Lyon,Liverpool,Norwich,Portimonense,Belenenses,West Ham,Man City,Bournemouth,Sheffield United,Burnley,Southampton,Crystal Palace,Everton,Watford,Brighton,Marseille,Reims,Santa Clara,Famalicao,Tottenham,Aston Villa,Angers,Bordeaux,Brest,Toulouse,Dijon,St Etienne,Montpellier,Rennes,Nice,Amiens,Gil Vicente,Porto,Benfica,Pacos Ferreira,Lille,Nantes,Leicester,Wolves,Newcastle,Arsenal,Strasbourg,Metz,Boavista,Aves,Man United,Chelsea,Maritimo,Sp Lisbon,Paris SG,Nimes,Sp Braga,Moreirense,Setubal,Tondela,Bayern Munich,Hertha,Ath Bilbao,Barcelona,Rio Ave,Dortmund,Augsburg,Freiburg,Mainz,Leverkusen,Paderborn,Werder Bremen,Fortuna Dusseldorf,Wolfsburg,FC Koln,Celta,Real Madrid,M'gladbach,Schalke 04,Valencia,Sociedad,Mallorca,Eibar,Leganes,Osasuna,Villarreal,Granada,Ein Frankfurt,Hoffenheim,Alaves,Levante,Union Berlin,RB Leipzig,Espanol,Sevilla,Guimaraes,Betis,Valladolid,Ath Madrid,Getafe,Parma,Juventus,Fiorentina,Napoli,Udinese,Milan,Cagliari,Brescia,Roma,Genoa,Sampdoria,Lazio,Spal,Atalanta,Torino,Sassuolo,Verona,Bologna,Inter,Lecce
0,1.488633,3.216128,1.600472,2.862259,3.476795,15.494452,1.824937,7.06939,0.973731,0.818897,3.142366,2.800964,4.79361,2.316075,1.061258,4.38076,10.704619,1.110831,2.75569,4.520224,11.859968,1.160365,1.377191,1.275245,1.760066,9.877412,1.439922,3.205667,2.184157,1.60284,2.083064,2.008778,1.560837,1.954129,3.82441,3.097125,2.012158,2.377607,3.451961,1.247523,2.302807,3.504557,2.290029,0.800199,1.958816,1.578311,2.84136,4.248904,2.939055,1.461049,2.592789,11.086367,12.616964,1.433531,3.749504,2.612491,7.410847,3.634363,1.599811,2.978979,2.610781,2.004343,2.146976,1.034098,4.491318,3.960429,1.701628,3.974625,16.893493,1.602116,5.055747,2.987222,2.012661,1.747229,15.423735,1.35381,3.242854,8.855389,3.905624,10.066645,1.63277,2.379648,1.298364,4.804906,1.103959,0.889788,1.063869,2.878471,2.333899,1.832723,8.118431,5.667382,2.53488,2.706166,4.044834,1.465059,1.660157,1.439479,2.389246,3.135679,2.811464,2.544404,2.134312,1.948523,2.106513,2.04144,11.525432,1.194199,3.915904,4.825911,2.385857,1.920674,3.84092,4.163742,2.928239,6.939066,2.375677,3.020675,1.476224,2.032266,2.859488,1.000015,5.153656,1.663672,1.509782,10.565933,1.236147,9.9776,1.400653,2.684517,2.881346,2.272698,7.30149,1.278843


In [273]:
df_Bund

,Bayern Munich,Hertha,Dortmund,Augsburg,Freiburg,Mainz,Leverkusen,Paderborn,Werder Bremen,Fortuna Dusseldorf,Wolfsburg,FC Koln,M'gladbach,Schalke 04,Ein Frankfurt,Hoffenheim,Union Berlin,RB Leipzig
0,15.423735,1.35381,10.066645,1.63277,2.379648,1.298364,4.804906,1.103959,0.889788,1.063869,2.878471,2.333899,5.667382,2.53488,2.544404,2.134312,2.04144,11.525432


In [220]:
#ptPS2=PS_Gdescent(ptPS,iterations=10000)
#df_PTA=pd.DataFrame(ptPS2[0],columns=cc2)
bundPS2=PS_Gdescent(bundPS,iterations=10000)
#df_Bund=pd.DataFrame(bundPS2[0],columns=cc3)
#llPS2=PS_Gdescent(llPS,iterations=10000)
#df_LL=pd.DataFrame(llPS2[0],columns=cc4)

In [219]:
#cc=findall_clubs('PLTeste',20)
#premPS_=PowerDF(prem,cc)
#cc2=findall_clubs('LPTeste',18)
ptPS=PowerDF(lp,cc2,H_coef=2)
#cc3=findall_clubs('BundTeste',18)
#bundPS=PowerDF(bund,cc3)
#ll=pd.read_excel('LLTeste.xlsx',0)
#cc4=findall_clubs('LLTeste',20)
#llPS=PowerDF(ll,cc4)